# PMSM

> Definition of motor class using stator and rotor subclasses.

In [ ]:
#| default_exp pmsm

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import sys
sys.path.append("..")

In [ ]:
#| export
import numpy as np
from PIL import Image

In [ ]:
from emdesigner.stator import stator
from emdesigner.rotor import rotor

### motor class is created using the following:

1. stator class
2. rotor class
3. radial airgap

In [ ]:
#| export
class pmsm(stator, rotor):
    def __init__(self, s = stator(), r = rotor()):
        self.stator = s
        self.rotor = r
        self.airgap = 0
        self.valid = False
    
    def valid_design(self):
        if self.airgap <= 0:
            print('motor airgap cannot be negative or zero')
            return 0
        
        self.valid = True
        
        


## test examples

define stator and rotor classes with corresponding values

In [ ]:
s1 = stator()
s1.slots = 12
s1.poles = 10
s1.outerdiameter = 125
s1.innerdiameter_fraction = 75/125
s1.slotopening_fraction = 0.3
s1.shoeheight_fraction = 0.1
s1.toothwidth_fraction = 0.5
s1.backiron_fraction = 0.3
s1.stacklength = 60
s1.windinglayers = 2
s1.coilpitch = 1
s1.turns = 11
s1.strands = 1
s1.strand_dia = 2
s1.parallelpaths = 2
s1.steel_grade = '50C470'

In [ ]:
r1 = rotor()
r1.poles = 10
r1.outerdiameter = 73
r1.innerdiameter_fraction = 25/73
r1.poleembrace = 0.83
r1.magnetthickness_fraction = 2*4/(73 - 25)
r1.stacklength = 60
r1.magnetgrade = 'N30'
r1.steel_grade = '50C470'
r1.valid_design()
r1.valid

True

### example 1:

create an instance of motor class using predefined stator and rotor classes

In [ ]:
m1 = pmsm(s1,r1)

In [ ]:
m1.rotor.outerdiameter

73

### example 2:

create an instance of motor class without any inputs. Assign a pre-existing stator and rotor class instances to the defined motor class instance

In [ ]:
m2 = pmsm()

In [ ]:
m2.rotor = r1
m2.stator = s1

In [ ]:
m2.rotor.outerdiameter

73

### example 3:

create an instance of motor class without any inputs. Assign stator and rotor values to the instance

In [ ]:
m3 = pmsm()

In [ ]:
m3.rotor.outerdiameter = 73

In [ ]:
m3.rotor.outerdiameter

73

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()